In [1]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from pprint import pprint

In [2]:
# сделаем рандомный UserAgent
ua = UserAgent()
ua.random

# ссылка соcтоит из основного сайа и индекса html
url_1 = 'https://books.toscrape.com/'
index_1 = '/index.html'

headers = {
    "User-Agent": ua.random} 

# запускаем сессию
session = requests.Session()

In [3]:
response = session.get(url_1+index_1, headers=headers)
if response.status_code < 400:
    soup = BeautifulSoup(response.text, 'html.parser')
else:
    print('Что-то пошло не так')


Если предыдущий код не выдал сообщение, значит, парсинг прошел успешно. <br>
Сначала составим список всех категорий книг

In [4]:
bc = soup.find('ul', { 'class': 'nav-list'}).findChildren('a')
books_category = {}
for el in bc[1:]:
    books_category[el.getText().strip()] = url_1 + el.get('href')

In [5]:
books_category

{'Travel': 'https://books.toscrape.com/catalogue/category/books/travel_2/index.html',
 'Mystery': 'https://books.toscrape.com/catalogue/category/books/mystery_3/index.html',
 'Historical Fiction': 'https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html',
 'Sequential Art': 'https://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html',
 'Classics': 'https://books.toscrape.com/catalogue/category/books/classics_6/index.html',
 'Philosophy': 'https://books.toscrape.com/catalogue/category/books/philosophy_7/index.html',
 'Romance': 'https://books.toscrape.com/catalogue/category/books/romance_8/index.html',
 'Womens Fiction': 'https://books.toscrape.com/catalogue/category/books/womens-fiction_9/index.html',
 'Fiction': 'https://books.toscrape.com/catalogue/category/books/fiction_10/index.html',
 'Childrens': 'https://books.toscrape.com/catalogue/category/books/childrens_11/index.html',
 'Religion': 'https://books.toscrape.com/catalogue/categor

Список категорий книг представляет собой словарь с парой ключ-значение: (Тема, ссылка на категорию)

Дальше попробуем спарсить все книги из каждой категории <br>
*пример такого кода можно посмотреть [тут](./parse.ipynb)*

In [6]:
all_books = {}

for category, url_category in books_category.items():

    books_in_category = []
    url_category = '/'.join(url_category.split('/')[:-1])
    index = 'index.html'

    while True:
        
        response = session.get(url_category+'/'+index, headers=headers)
        
        if response.status_code < 400:
            soup_page = BeautifulSoup(response.text, 'html.parser')

        books_list = soup_page.find_all('article', ('class', 'product_pod'))
        for i in books_list:
            book = {}
            # для записи книги берем ее название и цену
            book['name'] = i.find('h3').find('a').get('title')
            book['price'] = i.find('div', { 'class': 'product_price'}).find('p', { 'class': 'price_color'}).getText()[1:]
            # PS: хотел взять еще рейтинг, но пока не получилось
            # добавляем книгу в общим список
            books_in_category.append(book)
            all_books[category] = books_in_category
    
        try:
            index = soup_page.find('li', { 'class': 'next'}).find('a').get('href')
    
        except:
            break
    
    print(f'Категория {category} добавлена. В ней {len(all_books[category])} книг')



Категория Travel добавлена. В ней 11 книг
Категория Mystery добавлена. В ней 32 книг
Категория Historical Fiction добавлена. В ней 26 книг
Категория Sequential Art добавлена. В ней 75 книг
Категория Classics добавлена. В ней 19 книг
Категория Philosophy добавлена. В ней 11 книг
Категория Romance добавлена. В ней 35 книг
Категория Womens Fiction добавлена. В ней 17 книг
Категория Fiction добавлена. В ней 65 книг
Категория Childrens добавлена. В ней 29 книг
Категория Religion добавлена. В ней 7 книг
Категория Nonfiction добавлена. В ней 110 книг
Категория Music добавлена. В ней 13 книг
Категория Default добавлена. В ней 152 книг
Категория Science Fiction добавлена. В ней 16 книг
Категория Sports and Games добавлена. В ней 5 книг
Категория Add a comment добавлена. В ней 67 книг
Категория Fantasy добавлена. В ней 48 книг
Категория New Adult добавлена. В ней 6 книг
Категория Young Adult добавлена. В ней 54 книг
Категория Science добавлена. В ней 14 книг
Категория Poetry добавлена. В ней 19 

Все категории книг добавлены, можно посмотреть как это выглядит

In [7]:
pprint(all_books)

{'Academic': [{'name': 'Logan Kade (Fallen Crest High #5.5)',
               'price': '£13.12'}],
 'Add a comment': [{'name': 'The Torch Is Passed: A Harding Family Story',
                    'price': '£19.09'},
                   {'name': 'The Mindfulness and Acceptance Workbook for '
                            'Anxiety: A Guide to Breaking Free from Anxiety, '
                            'Phobias, and Worry Using Acceptance and '
                            'Commitment Therapy',
                    'price': '£23.89'},
                   {'name': 'The Art Forger', 'price': '£40.76'},
                   {'name': 'On a Midnight Clear', 'price': '£14.07'},
                   {'name': 'Judo: Seven Steps to Black Belt (an Introductory '
                            'Guide for Beginners)',
                    'price': '£53.90'},
                   {'name': 'Shobu Samurai, Project Aryoku (#3)',
                    'price': '£29.06'},
                   {'name': 'Modern Romance', 'price': '£

Запишем эти данные в json-файл

In [8]:
import json

with open('book.json', 'w', encoding='UTF-8') as file:
    json.dump(all_books, file, ensure_ascii=False)

Для удобства работы представим эти данные в pd.DataFrame()

In [9]:
import pandas as pd

# создаем пустой датафрейм
df_all = pd.DataFrame()
# делаем датафреймы по каждой категории
for el in all_books.keys():
    df = pd.DataFrame(all_books[el])
    # добавляем столбец "Категория"
    df['category'] = el
    # объединяем все в один датафрейм
    df_all = pd.concat([df_all, df])
    
# назначаем новые индексы
df_all.reset_index(inplace=True)
df_all = df_all.drop('index', axis=1)
df_all

,name,price,category
0,It's Only the Himalayas,£45.17,Travel
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,£49.43,Travel
2,See America: A Celebration of Our National Par...,£48.87,Travel
3,Vagabonding: An Uncommon Guide to the Art of L...,£36.94,Travel
4,Under the Tuscan Sun,£37.33,Travel
...,...,...,...
995,Why the Right Went Wrong: Conservatism--From G...,£52.65,Politics
996,Equal Is Unfair: America's Misguided Fight Aga...,£56.86,Politics
997,Amid the Chaos,£36.58,Cultural
998,Dark Notes,£19.19,Erotica


Вывод: Поработали со скрепингом страницы книжного магазина. Научились работать с каждой категорией по отдельности, делать выгрузку книг по этим категориям. В итоге получили данные по 1000 книг (название книги и ее цена), которые успешно записали в json-файл и преобразовали в датафрейм для дальнейшей работы